<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%203/3_6_Mehr_Daten%2C_bessere_Ergebnisse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
!pip3 install skorch

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from skorch import NeuralNetClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from torch import nn
import torch

In [ ]:
class NeuralNetModule(nn.Module):
    def __init__(self, num_inputs, num_units=20, nonlin=nn.ReLU()):
        super(NeuralNetModule, self).__init__()

        self.nonlin = nonlin
        self.dense0 = nn.Linear(num_inputs, num_units)
        self.dropout = nn.Dropout(0.2)
        self.dense1 = nn.Linear(num_units, num_units)
        self.output = nn.Linear(num_units, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X


In [ ]:
def evaluate_clf(test_X, labels, clf):
    predictions = clf.predict(test_X)
    report = classification_report(labels, predictions)
    print(report)

## Download Dataset 

### Manuell
via https://www.kaggle.com/columbine/imdb-dataset-sentiment-analysis-in-csv-format

### Via API

Hinzufügen der kaggle.json
Speichern als ~/.kaggle/kaggle.json auf Linux, OSX, oder andere UNIX-based Betriebssysteme und unter C:\Users<Windows-username>.kaggle\kaggle.json auf Windows

Siehe https://www.kaggle.com/docs/api oder https://github.com/Kaggle/kaggle-api
        
Beispiel:
~/kaggle/kaggle.json

{"username":"openHPI","key":"das_ist_der_key"}

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d columbine/imdb-dataset-sentiment-analysis-in-csv-format

In [ ]:
import zipfile
with zipfile.ZipFile("imdb-dataset-sentiment-analysis-in-csv-format.zip", 'r') as zip_ref:
    zip_ref.extractall("")

import os  
os.rename('Train.csv','sentiment.csv') 

In [ ]:
def read_and_parse_train_df(nrows):
    df = pd.read_csv("sentiment.csv", nrows=nrows)

    # preprocessing
    df["text"] = df["text"].apply(lambda x: x.lower())
    df["text"] = df["text"].apply(lambda x: x.replace("\'", ""))
    df["text"] = df["text"].apply(lambda x: BeautifulSoup(x).text)
    
    return df

def read_and_parse_test_df():
    df = pd.read_csv("sentiment.csv")
    df = df.tail(1000)

    # preprocessing
    df["text"] = df["text"].apply(lambda x: x.lower())
    df["text"] = df["text"].apply(lambda x: x.replace("\'", ""))
    df["text"] = df["text"].apply(lambda x: BeautifulSoup(x).text)
    
    return df

In [ ]:
test_df = read_and_parse_test_df()

# 3.6 Mehr Daten, bessere Ergebnisse

In [ ]:
for nrows in [100, 500, 1_000, 5_000, 10_000, 49_000]:
    print(f"#{nrows}")
    print("-" * 53)
    train_df = read_and_parse_train_df(nrows)
    vectorizer = TfidfVectorizer()
    train_X = vectorizer.fit_transform(train_df["text"]).astype(np.float32)
    test_X = vectorizer.transform(test_df["text"]).astype(np.float32)

    neural_net = NeuralNetClassifier(
        module=NeuralNetModule,
        module__num_inputs = len(vectorizer.vocabulary_),
        max_epochs=10,
        optimizer=torch.optim.Adam,
        iterator_train__shuffle=True,
        verbose=0
    )
    neural_net.fit(train_X, train_df['label'])

    evaluate_clf(test_X, test_df["label"], neural_net)
    print("\n"*2)